In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import xgboost as xgb

# load data
data = pd.read_csv('../data/merged_data.csv')

data.shape

(1917, 419)

### The workflow for the data model is as follows:
1. Extract dataframe for each data group
2. Consolidate into 2 dataframes for categorical and numerical data
3. Consolidate into 1 dataframe for all data
4. Create dataframes with target variables for regression and classification
5. Implement lighweight ML pipelines to evaluate how the data fits the model
6. Evaluate model performance and iterate on data model

### Review Numeric Data

,cows_predose,cows_postdose
0,11.0,6.0
1,8.0,1.0
2,8.0,5.0
3,11.0,9.0
4,0.0,0.0
...,...,...
1912,0.0,0.0
1913,17.0,12.0
1914,0.0,0.0
1915,0.0,0.0


In [8]:
# in this cell we will create a number of dataframes that will merge into dataset for ML

# first we will group numeric and cat columns that are numeric

# create rsa df, including cols for baseline assessment plus 4 weeks of treatment
rsa = data[[col for col in data.columns if 'rsa' in col]].iloc[:, :5]

# create df for meds df, including cols for baseline assessment plus 4 weeks of treatment
meds = data[[col for col in data.columns if 'med' in col]].iloc[:, :10]

# create df for tests df, including cols for baseline assessment plus 4 weeks of treatment
tests = data[[col for col in data.columns if 'test' in col]].iloc[:, :45]

# create df for survey df, including cols for baseline assessment plus 4 weeks of treatment
survey = data[[col for col in data.columns if 'survey' in col]].iloc[:, :20]

# create df for cows, including cols for baseline assessment
cows = data[[col for col in data.columns if 'cows' in col]]

# concat num features into num_df
num_df = pd.concat([rsa, meds, tests, cows], axis=1)

num_df

,rsa_week_0,rsa_week_1,rsa_week_2,rsa_week_3,rsa_week_4,meds_methadone_0,meds_buprenorphine_0,meds_methadone_1,meds_buprenorphine_1,meds_methadone_2,...,test_Amphetamines_4,test_Cannabinoids_4,test_Benzodiazepines_4,test_MMethadone_4,test_Oxycodone_4,test_Cocaine_4,test_Methamphetamine_4,test_Opiate300_4,cows_predose,cows_postdose
0,1.0,1.0,1.0,1.0,1.0,0.0,8.0,0.0,160.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,11.0,6.0
1,1.0,1.0,1.0,1.0,1.0,0.0,8.0,0.0,48.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,8.0,1.0
2,1.0,1.0,1.0,1.0,1.0,30.0,0.0,170.0,0.0,310.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,8.0,5.0
3,1.0,1.0,1.0,0.0,1.0,0.0,16.0,0.0,152.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,11.0,9.0
4,1.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,152.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1.0,0.0,0.0,0.0,0.0,110.0,0.0,270.0,0.0,250.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1913,1.0,1.0,1.0,1.0,1.0,0.0,8.0,0.0,160.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,12.0
1914,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,160.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0
1915,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,160.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [9]:
list(num_df.columns)

['rsa_week_0',
 'rsa_week_1',
 'rsa_week_2',
 'rsa_week_3',
 'rsa_week_4',
 'meds_methadone_0',
 'meds_buprenorphine_0',
 'meds_methadone_1',
 'meds_buprenorphine_1',
 'meds_methadone_2',
 'meds_buprenorphine_2',
 'meds_methadone_3',
 'meds_buprenorphine_3',
 'meds_methadone_4',
 'meds_buprenorphine_4',
 'test_Propoxyphene_0',
 'test_Amphetamines_0',
 'test_Cannabinoids_0',
 'test_Benzodiazepines_0',
 'test_MMethadone_0',
 'test_Oxycodone_0',
 'test_Cocaine_0',
 'test_Methamphetamine_0',
 'test_Opiate300_0',
 'test_Propoxyphene_1',
 'test_Amphetamines_1',
 'test_Cannabinoids_1',
 'test_Benzodiazepines_1',
 'test_MMethadone_1',
 'test_Oxycodone_1',
 'test_Cocaine_1',
 'test_Methamphetamine_1',
 'test_Opiate300_1',
 'test_Propoxyphene_2',
 'test_Amphetamines_2',
 'test_Cannabinoids_2',
 'test_Benzodiazepines_2',
 'test_MMethadone_2',
 'test_Oxycodone_2',
 'test_Cocaine_2',
 'test_Methamphetamine_2',
 'test_Opiate300_2',
 'test_Propoxyphene_3',
 'test_Amphetamines_3',
 'test_Cannabinoids_

### Review Categorical Data

In [10]:
# next we will group the cat columns with text strings and one hot encode them

# demographic data
dem = data[[col for col in data.columns if col.startswith('dem_')]]

# create df for diagnosis
dsm = data[[col for col in data.columns if col.startswith('dsm_')]]

# create df for medical history
mdh = data[[col for col in data.columns if col.startswith('mdh_')]]

# create df for physical exam
pex = data[[col for col in data.columns if col.startswith('pex_')]]

# concat cat features into cat_df
cat_df = pd.concat([dsm, mdh, pex], axis=1)

# one hot encode cat_df
cat_df = pd.get_dummies(cat_df, dtype=int)

cat_df



,dsm_cannabis_abuse,dsm_cannabis_dependence,dsm_cannabis_no_diagnosis,dsm_cannabis_not_evaluated,dsm_cannabis_not_present,dsm_cocaine_abuse,dsm_cocaine_dependence,dsm_cocaine_no_diagnosis,dsm_cocaine_not_evaluated,dsm_cocaine_not_present,...,pex_head_neck_not_evaluated,pex_head_neck_not_present,pex_cardio_abnormal,pex_cardio_normal,pex_cardio_not_evaluated,pex_cardio_not_present,pex_skin_abnormal,pex_skin_normal,pex_skin_not_evaluated,pex_skin_not_present
0,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
3,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,0
4,0,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
1913,0,0,1,0,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,0,0
1914,0,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
1915,0,0,1,0,0,1,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1


### Create 2 dataframes for classification and regression models
1. Classification Dataframe - will use the `dropout` column as the target
2. Regression Dataframe - will use the NTR (negative test rate) column as the target

In [11]:
# ceate variable for target, which is the negative test rate
ntr = data['NTR']
dropout = data['dropout']
#responder = data['responder']
 
# merge df num, cat and ntr
classification_df = pd.concat([num_df, cat_df, dropout], axis=1)

regression_df = pd.concat([num_df, cat_df, ntr], axis=1)

classification_df.shape, regression_df.shape

((1917, 194), (1917, 194))

In [13]:
classification_df

,rsa_week_0,rsa_week_1,rsa_week_2,rsa_week_3,rsa_week_4,meds_methadone_0,meds_buprenorphine_0,meds_methadone_1,meds_buprenorphine_1,meds_methadone_2,...,pex_head_neck_not_present,pex_cardio_abnormal,pex_cardio_normal,pex_cardio_not_evaluated,pex_cardio_not_present,pex_skin_abnormal,pex_skin_normal,pex_skin_not_evaluated,pex_skin_not_present,dropout
0,1.0,1.0,1.0,1.0,1.0,0.0,8.0,0.0,160.0,0.0,...,0,0,1,0,0,0,1,0,0,1.0
1,1.0,1.0,1.0,1.0,1.0,0.0,8.0,0.0,48.0,0.0,...,0,0,1,0,0,1,0,0,0,1.0
2,1.0,1.0,1.0,1.0,1.0,30.0,0.0,170.0,0.0,310.0,...,0,0,1,0,0,1,0,0,0,1.0
3,1.0,1.0,1.0,0.0,1.0,0.0,16.0,0.0,152.0,0.0,...,0,0,1,0,0,0,1,0,0,1.0
4,1.0,0.0,0.0,0.0,0.0,0.0,16.0,0.0,152.0,0.0,...,1,0,0,0,1,0,0,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1912,1.0,0.0,0.0,0.0,0.0,110.0,0.0,270.0,0.0,250.0,...,0,0,1,0,0,1,0,0,0,0.0
1913,1.0,1.0,1.0,1.0,1.0,0.0,8.0,0.0,160.0,0.0,...,0,0,1,0,0,1,0,0,0,1.0
1914,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,160.0,0.0,...,0,1,0,0,0,0,1,0,0,0.0
1915,1.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,160.0,0.0,...,1,0,0,0,1,0,0,0,1,0.0


In [12]:
# save dfs to csv
classification_df.to_csv('../data/classification_df.csv', index=False)
regression_df.to_csv('../data/regression_df.csv', index=False)

### Run a simple regression model


In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

X, y = regression_df.drop('NTR', axis=1), regression_df['NTR']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=10, max_depth=3)

xg_reg.fit(X_train, y_train)

preds = xg_reg.predict(X_test)

from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np

# Assuming regression_df is your DataFrame and 'NTR' is the target variable
X, y = classification_df.drop('dropout', axis=1), classification_df['dropout']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Use XGBClassifier for classification
xg_clf = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, max_depth=3)

xg_clf.fit(X_train, y_train)

# Predict probabilities instead of classes for AUC calculation
pred_probs = xg_clf.predict_proba(X_test)[:, 1]

# Calculate AUC
auc_score = roc_auc_score(y_test, pred_probs)
print("AUC: %f" % (auc_score))

In [ ]:
# print classifcation report for training set
from sklearn.metrics import classification_report

y_pred = xg_clf.predict(X_train)
print(classification_report(y_train, y_pred))


In [ ]:
# print classification report for test set
from sklearn.metrics import classification_report
y_pred = xg_clf.predict(X_test)
print(classification_report(y_test, y_pred))

### Plot tree to see the structure of the decision tree

In [ ]:
import xgboost as xgb
from graphviz import Source

# Assuming 'xg_reg' is your trained XGBoost model
graph = xgb.to_graphviz(xg_clf, num_trees=9)
# can I print the num trees argument?

# Display the tree plot directly in the notebook
graph

### Hyperparameter tuning
The parameters that can be tuned are significantly different for each base learner<br>
<br>
For the tree based learner -  Most frequently tuned parameters are outlined below.<br>
<br>
<u>learning rate</u> - Effects how quickly the model fits the residual error using additional base learners<br>
<br>
A low learning rate will require more boosting rounds to achieve the same reduction in residual error, compared to a model with a high learning rate<br>
<br>

<u>Regularization</u> - Applies penalty to prediction errors within a certain threshold, to reduce overfitting<br>
<u>gamma</u> - min loss reduction to create new tree split<br>
<u>lambda</u> - L2 regularization on leaf weights<br>
<u>alpha</u> - L1 regularization on leaf weights<br>
<br>To learn more about L1 and L2 regularization, please read the following      [article](https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c)<br>
<br>
<u>max_depth</u> - positive integer value - effects how deeply each tree is allowed to grow during any given boosting round<br>
<br>
<u>subsample</u> - value between 0 - 1, fraction of the training set that can be used for any giving boosting round.  Low and high values can lead to under and overfitting<br>
<br>
<u>colsample_bytree</u> - fraction of features you can select from during any given boosting round and must also be a value between 0 and 1<br><br>
A large value means that almost all features can be used to build a tree, during a given boosting round, where as a small value means that as the fraction  of features  that can be selected from is very small.  In general smaller colsamplebytree values, behaves as additional regularization to the model.  Where using all columns in certain cases can overfit a trained model.<br>
<br>
For the linear based learner the number of tunable parameters is much smaller, L1 and L2 regularization for the weights, model bias<br><br>
Number of estimators is a tunable parameter
The parameters that can be tuned are significantly different for each base learner
<br>

For the linear based learner the number of tunable parameters is much smaller, L1 and L2 regularization for the weights, model bias
Number of estimators is a tunable parameter

### Learning Rate

In [ ]:
# hyperparameter tuning - Learning Rate

# define X, y
X, y = regression_df.drop('NTR', axis=1), regression_df['NTR']

# Create dmatrix with oud clinical data
oud_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary for each tree (boosting round)
params = {"objective":"reg:linear", "max_depth":3}

# Create list of eta values and empty list to store final round rmse per xgboost model
eta_vals = [0.001, 0.01, 0.1, 0.2, 0.3]
best_rmse = []

# Systematically vary the eta 
for curr_val in eta_vals:

    params["eta"] = curr_val
    
    # Perform cross-validation: cv_results
    cv_results = xgb.cv(dtrain=oud_dmatrix, params=params, nfold=3,
                    num_boost_round=10, early_stopping_rounds=5,
                    metrics="rmse", as_pandas=True, seed=123)
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(eta_vals, best_rmse)), columns=["eta","best_rmse"]))

### Max Depth

In [ ]:
# parameter tuning max_depth

# Create your housing DMatrix
oud_dmatrix = xgb.DMatrix(data=X,label=y)

# Create the parameter dictionary
params = {"objective":"reg:linear"}

# Create list of max_depth values
max_depths = [2, 5, 10, 20]
best_rmse = []

# Systematically vary the max_depth
for curr_val in max_depths:

    params["max_depth"] = curr_val
    
    # Perform cross-validation
    cv_results = xgb.cv(dtrain=oud_dmatrix, params=params, nfold=2,
                 num_boost_round=10, early_stopping_rounds=5,
                 metrics="rmse", as_pandas=True, seed=123)
    
    
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(max_depths, best_rmse)),columns=["max_depth","best_rmse"]))

### Col Sample By Tree

In [ ]:
# parameter tuning colsample_bytree

# Create your housing DMatrix
oud_dmatrix = xgb.DMatrix(data=X,label=y)

# Create the parameter dictionary
params={"objective":"reg:linear","max_depth":3}

# Create list of hyperparameter values: colsample_bytree_vals
best_rmse = []
colsample_bytree_vals = [0.1, 0.5, 0.8, 1]

# Systematically vary the hyperparameter value 
for curr_val in colsample_bytree_vals:
 
    params['colsample_bytree'] = curr_val
    
    # Perform cross-validation
    cv_results = xgb.cv(dtrain=oud_dmatrix, params=params, nfold=2,
                 num_boost_round=10, early_stopping_rounds=5,
                 metrics="rmse", as_pandas=True, seed=123)
    
    # Append the final round rmse to best_rmse
    best_rmse.append(cv_results["test-rmse-mean"].tail().values[-1])

# Print the resultant DataFrame
print(pd.DataFrame(list(zip(colsample_bytree_vals, best_rmse)), columns=["colsample_bytree","best_rmse"]))

### GridsearchCV

Review of grid search and random search
How do we find the optimal values for several hyper-parameters simultaneously?

Leading to the lowest lost possible when their values interact in non-obvious, non-linear ways.

2 common strategies are gridsearch and random search - import to review advantages and disadvantages, looking at examples
of how it can be used with xgboost and sci-kit learn packages - 

Gridsearch method of exhaustively searching through a collection of possible parameter values.
For example if you have 2 hyperparameters you would like to tune and 4 possible values for each hp,
a gridsearch for that parameter space will try all 16 of possible parameter configurations.

In a grid search you try every parameter configuration, evaluate a metric for that configuration and give you the best
value for the metric you were using, in our case the rmse.
How do we find the optimal values for several hyper-parameters simultaneously

Leading to the lowest lost possible when their values interact in non-obvious, non-linear ways

2 common strategies are gridsearch and random search - import to review advantages and disadvantages, looking at examples
of how it can be used with xgboost and sci-kit learn packages - 

Gridsearch method of exhaustively searching through a collection of possible parameter values
For example if you have 2 hyperparameters you would like to tune and 4 possible values for each hp
a gridsearch  for that parameter space will try all 16 of possible parameter configurations

In a grid search you try every parameter configuration, evaluate a metric for that configuration and give you the best
value for the metric you were using, in our case the rmse


# Regression

In [ ]:
from sklearn.model_selection import GridSearchCV

# prepare the dmatrix
#dtrain = xgb.DMatrix(X, label=y)
X, y = regression_df.drop('NTR', axis=1), regression_df['NTR']
regression_dmatrix = xgb.DMatrix(data=X, label=y)

gbm_param_grid = {'learning_rate':[0.01, 0.1, 0.5, 0.9],
                  'n_estimators': [200],
                  'subsample': [0.3, 0.5, 0.9]}
gbm = xgb.XGBRegressor()
grid_mse = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, scoring='neg_mean_squared_error', cv=4, verbose=1)

grid_mse.fit(X, y)
print("Best parameters found: ", grid_mse.best_params_)
print("Lowest RMSE found: ", np.sqrt(np.abs(grid_mse.best_score_)))

# Classification

In [ ]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import numpy as np



# create train, test and eval set
from fast_ml.model_development import train_valid_test_split
train = 0.7
test = 0.2
validation = 0.1
X_train, y_train, X_val, y_val, X_test, y_test = train_valid_test_split(classification_df,
                                                                             target='dropout',
                                                                             train_size=train,
                                                                             test_size=test,
                                                                             valid_size=validation)


gbm_param_grid = {
	'learning_rate': [0.01, 0.1, 0.5, 0.9],
	'n_estimators': [200, 300, 400],
	'subsample': [0.3, 0.5, 0.9],
    'max_depth': [3, 6, 9],
	'objective': ['binary:logistic']  # Specify binary logistic objective
}

gbm = xgb.XGBClassifier(use_label_encoder=False)  # XGBClassifier for classification tasks
grid_auc = GridSearchCV(estimator=gbm, param_grid=gbm_param_grid, scoring='roc_auc', cv=4, verbose=1, return_train_score=True)  # Use ROC AUC as the scoring metric

grid_auc.fit(X_train, y_train)
print("Best parameters found: ", grid_auc.best_params_)
print("Highest ROC AUC found: ", grid_auc.best_score_)

In [ ]:
# refit the model with the best params
best_params = grid_auc.best_params_
best_gbm = xgb.XGBClassifier(**best_params)
best_gbm.fit(X_train, y_train, eval_set=[(X_train,y_train),(X_val, y_val)], eval_metric=['auc','error'], verbose=False)

In [ ]:
# plot the confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
y_pred = best_gbm.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=best_gbm.classes_)
disp.plot()

In [ ]:
# retrieve performance metrics
results = best_gbm.evals_result()
epochs = len(results['validation_0']['auc'])
x_axis = range(0, epochs)

# plot log loss and classification error 
fig, axs = plt.subplots(ncols=2, figsize=(18, 5))

# plot log loss
axs[0].plot(x_axis, results['validation_0']['error'], label='Train')
axs[0].plot(x_axis, results['validation_1']['error'], label='Test')
axs[0].legend()
axs[0].set_ylabel('Error')
axs[0].set_title('XGBoost Error', fontsize=16)

# plot classification error
axs[1].plot(x_axis, results['validation_0']['auc'], label='Train')
axs[1].plot(x_axis, results['validation_1']['auc'], label='Test')
axs[1].legend()
axs[1].set_ylabel('AUC')
axs[1].set_title('XGBoost AUC', fontsize=16)

plt.show()


### Evaluate the fitted model

In [ ]:
# Print the classification report
from sklearn.metrics import classification_report, accuracy_score
y_pred_train = best_gbm.predict(X_train)
y_pred_val = best_gbm.predict(X_val)
y_pred_test = best_gbm.predict(X_test)

# print accuracy for train, val and test
print("Train Accuracy: ", accuracy_score(y_train, y_pred_train))
print("Validation Accuracy: ", accuracy_score(y_val, y_pred_val))
print("Test Accuracy: ", accuracy_score(y_test, y_pred_test))


In [ ]:
# print classification report for training
print('Train Classification Report')
print(classification_report(y_train, y_pred_train))

In [ ]:
# Print the classification report for test
print("Test classification report:")
print(classification_report(y_test, y_pred_test))

### Plot the AUC for training and Test

### Plot feature importance for classification

In [ ]:
#plot tree
from xgboost import plot_tree
import matplotlib.pyplot as plt


In [ ]:
# plot feature importance
xgb.plot_importance(grid_auc.best_estimator_, importance_type='weight', max_num_features=15, title='Feature Importance - Weight', xlabel='F Score', ylabel='Features', color='lightgreen', grid=False)
plt.show()

In [ ]:
# Assuming 'xg_reg' is your trained XGBoost model
graph = xgb.to_graphviz(grid_auc.best_estimator_, num_trees=0)
# can I print the num trees argument?

# Display the tree plot directly in the notebook
graph